In [1]:
from flair.data import Sentence
from flair.models import SequenceTagger
import pandas as pd
import numpy as np
import time
import re

pd.set_option('display.max_colwidth', None)

c:\Users\samue\anaconda3\envs\cdas_o22\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#paths
pathBHR = '../archivos/csv/Bot_history_report.csv'
pathPrueba1 = '../data/Prueba 1 ITAM.csv'
pathPalabrasLlave = '../archivos/csv/Copia de Palabras llave RFP.csv'
pathTraining = '../archivos/csv/Training_data.csv'
pathOutput = '../out/out2.csv'
pathRes = '../out/res2.csv'
pathDur = '../out/dur2.csv'


In [3]:
#%% cositas de archivos
df = pd.read_csv(pathPrueba1)
print(df)
serie = list(df['Descripción'])
f = open(pathOutput, "w", encoding='utf8');f.write('num,label,name,start_pos,end_pos,proba' + '\n');f.close() #reiniciamos nuestro archivo de output
f = open(pathOutput, "a", encoding='utf8')


   Fecha y hora de fin                   Propietario  \
0           01/08/2022        Juana Alvarado Paredes   
1           05/08/2022   Germán César Calixto Cortéz   
2           04/08/2022         César Castilla Flores   
3           08/08/2022  Carlos Antonio Campi Aguirre   
4           08/08/2022        Juana Alvarado Paredes   
5           08/08/2022         César Castilla Flores   
6           10/08/2022         César Castilla Flores   
7           15/08/2022  Rosario Adriana López Zavala   
8           23/08/2022         César Castilla Flores   
9           22/08/2022   Germán César Calixto Cortéz   
10          25/08/2022      Ricardo Nájera Mondragón   
11          25/08/2022      Ricardo Nájera Mondragón   
12          29/08/2022        Rodrigo Fuentes Patiño   
13          29/08/2022      Ricardo Nájera Mondragón   
14          02/09/2022         César Castilla Flores   
15          01/09/2022      Ricardo Nájera Mondragón   
16          30/08/2022         César Castilla Fl

In [4]:
# aplicamos el modelo que nos da una lista de entidades nombradas y escribimos al archivo
startTime = time.time() 
tagger = SequenceTagger.load("flair/ner-spanish-large") # load tagger
for index, sen in enumerate(serie):
    sentence = Sentence(sen)
    tagger.predict(sentence) # predict NER tags
    for entity in sentence.get_spans('ner'): # iterate over entities and print
        f.write(str(index) + ',' +  str(entity.get_label("ner").value) + ',' + str(entity.text) + ',' + str(entity.start_position) + ',' +  str(entity.end_position) + ',' +  str(entity.get_label("ner").score) + '\n')
f.close()

endTime = time.time()
duration = endTime - startTime # get the execution time

Downloading: 100%|██████████| 2.24G/2.24G [02:25<00:00, 15.4MB/s]

2023-01-17 17:24:19,117 loading file C:\Users\samue\.flair\models\ner-spanish-large\models--flair--ner-spanish-large\snapshots\9d4671d2f345c1258f37a29ce2321067f2ed296e\pytorch_model.bin



c:\Users\samue\anaconda3\envs\cdas_o22\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\samue\.flair\models\ner-spanish-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%|██████████| 616/616 [00:00<?, ?B/s] 
Downloading: 100%|██████████| 5.07M/5.07M [00:02<00:00, 2.47MB/s]
Down

2023-01-17 17:24:43,413 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-LOC, S-ORG, B-PER, I-PER, E-PER, S-MISC, B-ORG, E-ORG, S-PER, I-ORG, B-LOC, E-LOC, B-MISC, E-MISC, I-MISC, I-LOC, <START>, <STOP>


In [5]:
with open(pathDur, 'w') as the_file:
    the_file.write(str(duration) + " seconds")
    
serie1 = serie.copy()

In [6]:
#%% reemplazamos las palabras de regex por #####
def testar_regex(texto):

    #correo electronico
    regex=re.sub("([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)", '#####', texto)

    #clave de elector
    regex=re.sub("[A-Z]{6}[0-9]{6,}(H|M)[0-9]{3}", '#####', regex)

    #curp
    regex=re.sub("[A-Z]{4}[0-9]{6}[A-Z]{6}([A-Z][0-9]|0[0-9])", '#####', regex)

    #RFC
    regex=re.sub("[A-Z,Ñ,&]{3,4}([0-9]{2})(0[1-9]|1[0-2])(0[1-9]|1[0-9]|2[0-9]|3[0-1])[A-Z|\d]{3}", '#####', regex)

    #numero cartilla militar
    regex=re.sub("(A|B|C)([0-9]{7,8}|\-[0-9]{7,8})", '#####', regex)

    #numero tel
    regex=re.sub("[0-9]{8,}|[0-9]{3} [0-9]{7}|[0-9]{2,3}(\-|.)[0-9]{4}(\-|.)[0-9]{4}", '#####', regex)

    return regex

In [7]:
#%% reemplazamos las palabras del modelo por #####
df2 = pd.read_csv(pathOutput)
for j in range(len(serie)):
    for i in list(df2.query('num=='+str(j))['name']):
        serie1[j] = serie1[j].replace(i, '### ')
        serie1[j] = testar_regex(serie1[j])



In [8]:
#%% construimos el csv
df3 = pd.DataFrame(list(zip(serie, serie1)), columns = ['original', 'censurado'])
df3.to_csv(pathRes, index=False, encoding='utf-8')
df3

,original,censurado
0,"La que suscribe Sandra Huerta con número de seguridad social 72795608040 por medio del presente me permito solicitar una cita para que hacer trámite administrativo para que se me reconozcan las semanas cotizadas en la Empresa Proyectos Marinos, S.C., Registro B10-11696-10-6, ubicada en la Calle de Ibsen No. 40 piso 7, en la Colonia Polanco, Alcaldía Miguel Hidalgo, CDMX, C.P. 11550M mi periodo laborado del 03 de Enero de 1980 al 15 de febrero de 198","La que suscribe ### con número de seguridad social ##### por medio del presente me permito solicitar una cita para que hacer trámite administrativo para que se me reconozcan las semanas cotizadas en la ### , ### , Registro ### , ubicada en la ### piso 7, en la ### , ### , ### , C.P. 11550M mi periodo laborado del 03 de Enero de 1980 al 15 de febrero de 198"
1,"Me refiero a su correo electrónico, recibido en el Centro de Atención a la Sociedad (CAS) de este Instituto, en el cual manifiesta que requiere acceso a su expediente medico de la atención que le brindaron a su familiar por parte de una psicóloga.","Me refiero a su correo electrónico, recibido en el ### (### ) de este ### , en el cual manifiesta que requiere acceso a su expediente medico de la atención que le brindaron a su familiar por parte de una psicóloga."
2,"Me dirijo a ustedes de la manera más atenta para solicitar copia de estudios que se me realizaron en el Hospítal 20 de Noviembre del ISSSTE, según carta anexa. El día de hoy 2 de agosto acudí al hospital antes mencionado y me comentaron que ya no se solicitan ahí, me dieron la dirección de ustedes, llegué y la chiuca que me atendió en una recepción del CAS me informó que no tenía conocimiento alguno, es por eso que decidí enviarles un correo con mi solicitud, misma que agradeceré confirmar su recepción y en qué tiempo puedo contar con ellos. Agradezco su atención","Me dirijo a ustedes de la manera más atenta para solicitar copia de estudios que se me realizaron en el ### del ### , según carta anexa. El día de hoy 2 de agosto acudí al hospital antes mencionado y me comentaron que ya no se solicitan ahí, me dieron la dirección de ustedes, llegué y la chiuca que me atendió en una recepción del ### me informó que no tenía conocimiento alguno, es por eso que decidí enviarles un correo con mi solicitud, misma que agradeceré confirmar su recepción y en qué tiempo puedo contar con ellos. Agradezco su atención"
3,"""...Soy Lic. en Ciencias Políticas y Administración Pública, egresado por la Universidad Iberoamenricana Campus Puebla, tengo 10 años de experiencia dentro de la Administración Pública municipal y Estatal, trabajé cuatro años y medio en la atención de solicitudes de acceso de información y protección de datos personales, así como en la administración de información del Portal de Transparencia de Puebla Comunicaciones y fui Titular de la Unidad de Transparencia del Centro de Conciliación Laboral del Estado de Puebla. El motivo de este mensaje es porque no he logrado cargar mi información curricular en la página web del INAI, motivo por el cual solicito de su valioso apoyo para poder ingresar mi información ya qeu me encantaría poder sumarme a su proyecto y ser parte del Instituto Nacional de Transparencia. Espero de ante mano poder ser considerado ya que tengo mucha ilusión de poder formar parte del gran equipo que conforman. Muchas gracias, les envío un cordial saludo. Atentamente..."" (sic)","""...Soy ### . en ### y ### , egresado por la ### , tengo 10 años de experiencia dentro de la ### municipal y ### , trabajé cuatro años y medio en la atención de solicitudes de acceso de información y protección de datos personales, así como en la administración de información del ### y fui ### del ### . El motivo de este mensaje es porque no he logrado cargar mi información curricular en la página web del ### , motivo por el cual solicito de su valioso apoyo para poder ingresar mi información ya qeu me encantaría poder sumarme a su proyecto y ser pa